### Import the library and instantiate the 2 classes

In [0]:
import torch
import torch.nn as nn

In [0]:
pool = nn.MaxPool2d(2, stride=2, return_indices=True)
unpool = nn.MaxUnpool2d(2, stride=2)

---

### MaxPool

In [0]:
x = torch.tensor([[[[1.5000,   1.7000,   1.4000,   1.3000], 
                    [2.0000,   2.1000,   1.8000,   1.6000], 
                    [2.3000,   1.9000,   1.5000,   1.4000], 
                    [2.2000,   2.1000,   1.6000,   1.7000]]]], requires_grad=True) # 4 Rows, 4 columns

In [0]:
print('INPUT for MaxPool: \n')
x

INPUT for MaxPool: 



tensor([[[[1.5000, 1.7000, 1.4000, 1.3000],
          [2.0000, 2.1000, 1.8000, 1.6000],
          [2.3000, 1.9000, 1.5000, 1.4000],
          [2.2000, 2.1000, 1.6000, 1.7000]]]], requires_grad=True)

In [0]:
out, idxs = pool(x)

In [0]:
# this is the output from Forward() of MaxPool
print('OUTPUT from MaxPool: \n')
print(out)

OUTPUT from MaxPool: 

tensor([[[[2.1000, 1.8000],
          [2.3000, 1.7000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)


In [0]:
# these are the poolingIndices
print('OUTPUT INDICES from MaxPool: \n')
print(idxs)

OUTPUT INDICES from MaxPool: 

tensor([[[[ 5,  6],
          [ 8, 15]]]])


In [0]:
# this is gy for maxpooling layer simulated with random values
gy = torch.tensor([[[[1., 2.], [3., 4.]]]])
print('Hypothetical gy for MaxPool: \n')
print(gy)

Hypothetical gy for MaxPool: 

tensor([[[[1., 2.],
          [3., 4.]]]])


In [0]:
out.backward(gy)

In [0]:
# this is g for maxpooling layer
g = x.grad
print('g for MaxPool: \n')
print(g)

g for MaxPool: 

tensor([[[[0., 0., 0., 0.],
          [0., 1., 2., 0.],
          [3., 0., 0., 0.],
          [0., 0., 0., 4.]]]])


---

### UnPool

In [0]:
# input of unpool is output of maxpool
# pooling Indices are the indices returned by maxpool
input = torch.tensor([[[[2.1000, 1.8000], [2.3000, 1.7000]]]], requires_grad=True)
poolingIndices = torch.tensor([[[[ 5,  6], [ 8, 15]]]])

In [0]:
print('INPUT for UnPool: \n')
print(input)

INPUT for UnPool: 

tensor([[[[2.1000, 1.8000],
          [2.3000, 1.7000]]]], requires_grad=True)


In [0]:
# this is different from mlpack because mlpack follows column-major ordering 
# while pytorch follows row major ordering.
# the indices in mlpack are  [5, 9, 2, 15] 
# the indices in pytorch are [5, 6, 8, 15]
print('INPUT INDICES for UnPool: \n')
print(poolingIndices)

INPUT INDICES for UnPool: 

tensor([[[[ 5,  6],
          [ 8, 15]]]])


In [0]:
output = unpool(input, poolingIndices)

In [0]:
# this is the output from Forward() of UnPool in mlpack
print('OUTPUT from UnPool: \n')
print(output)

OUTPUT from UnPool: 

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 2.1000, 1.8000, 0.0000],
          [2.3000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 1.7000]]]], grad_fn=<MaxUnpool2DBackward>)


In [0]:
# this is gy in mlpack for the unpooling layer. Here it is filled with random values,= just for simulation.
# this tensor represents the upstream derivative dL/doutput, 
# where L represents the output of the loss function at the end of the network.
gy = torch.tensor([[[[ 1.,  2,  3,  4], [ 5,  6,  7,  8], [ 9, 10, 11, 12], [13, 14, 15, 16]]]])
print('Hypothetical gy for UnPool: \n')
print(gy)

Hypothetical gy for UnPool: 

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])


In [0]:
output.backward(gy)

In [0]:
# this is g in mlpack for the unpooling layer for the above simulated values of gy.
# this represents the derivative [dL/dinput = (doutput/dinput) * (dL/doutput)]
g = input.grad
print('g for UnPool: \n')
print(g)

g for UnPool: 

tensor([[[[ 6.,  7.],
          [ 9., 16.]]]])


---